In [1]:
# from data_multi import Bandit_multi
# from learner_diag import NeuralUCBDiag
import numpy as np
import argparse
import pickle
import os
import time
import torch
import numpy as np
from scipy.stats import truncnorm
import matplotlib.pyplot as plt

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
device
# torch.set_num_threads(8)
# torch.set_num_interop_threads(8)



def truncated_gaussian(mean, variance, a, b, size):
    # Calculate the standard deviation from the variance
    std_dev = np.sqrt(variance)

    # Calculate the lower and upper bounds for truncation
    lower_bound = (a - mean) / std_dev
    upper_bound = (b - mean) / std_dev

    # Generate samples from the truncated normal distribution
    samples = truncnorm.rvs(lower_bound, upper_bound, loc=mean, scale=std_dev, size=size)

    return np.array(samples)

def reshape_context(context, A):
    d = context.shape[0]
    reshaped_context = np.zeros((A, d * A))
    
    for i in range(A):
        start_idx = i * d
        end_idx = start_idx + d
        reshaped_context[i, start_idx:end_idx] = context
    
    return reshaped_context

class Evaluation:

    def __init__(self, game_name, task, n_folds, horizon, game, label, context_type):
        self.game_name = game_name
        self.task = task
        self.n_folds = n_folds
        self.horizon = horizon
        self.game = game
        self.label =  label
        self.context_type = context_type
        

    def get_outcomes(self, game, ):
        outcomes = np.random.choice( game.n_outcomes , p= list( game.outcome_dist.values() ), size= self.horizon) 
        return outcomes

    def get_feedback(self, game, action, outcome):
        return game.FeedbackMatrix[ action ][ outcome ]

    def eval_policy_once(self, alg, game, job):

        alg.reset()

        context_generator, jobid = job

        np.random.seed(jobid)

        # context,distribution = context_generator.get_context()
        # outcome = np.random.choice( 2 , p = distribution )
        # context = reshape_context(context, alg.A)

        cumRegret =  np.zeros(self.horizon, dtype =float)
        compteur = None
        for t in range(self.horizon):
            compteur = t
            if t % 1000 == 0 :
                print(t)

            context,distribution = context_generator.get_context()
            outcome = np.random.choice( 2 , p = distribution )
            context = reshape_context(context, alg.A)
            
            action = alg.get_action(t, context)

            print('t', t, 'action', action, 'outcome', outcome, 'gaps', ( game.LossMatrix[0,...] - game.LossMatrix[1,...])  @ distribution  )
            print()
            feedback =  self.get_feedback( game, action, outcome )

            alg.update(action, feedback, outcome, t, context )

            i_star = np.argmin(  [ game.LossMatrix[i,...] @ np.array( distribution ) for i in range(alg.N) ]  )
            loss_diff = game.LossMatrix[action,...] - game.LossMatrix[i_star,...]
            val = loss_diff @ np.array( distribution )
            cumRegret[t] =  val

        return cumRegret
    
class ToyContexts:

    def __init__(self, ):
        self.type = 'toy'
        self.d_context = 2

    def get_context(self, ):
        p = 1 if np.random.uniform(0, 1)>0.5 else 0 
        return np.array([1,p]), [p, 1-p]


In [ ]:
d = 10
sigma_i = 2
lbd = 0.05
value1, value2, value3, value4 = [], [], [], []
for t in range(1, 10000):
    value1.append(  d * (  np.sqrt(  d * np.log(t) + 2 * np.log(1/t**2)   ) + 2 ) )
    value2.append(  d * (  np.sqrt(  (d+1) * np.log(t)   ) + 2 ) )
    value3.append(  sigma_i * (  np.sqrt(  d * np.log(t) + 2 * np.log(1/t**2)   ) +  sigma_i ) )
    value4.append(  sigma_i * (  np.sqrt(  d * np.log(t) + 2 * np.log(1/t**2)   ) + np.sqrt(lbd) * sigma_i ) )

plt.plot( range(9999), value1, label = 'width1')
plt.plot( range(9999), value2, label = 'width2')
plt.plot( range(9999), value3, label = 'width3')
plt.plot( range(9999), value4, label = 'width4')
plt.legend()

In [ ]:
p = 435
sigma_i = 2
lbd = 0.05
value3, value4 = [], []
for t in range(1, 10000):
    value3.append(  sigma_i * (  np.sqrt(  p * np.log(t) + 2 * np.log(1/t**2)   ) +  sigma_i ) )
    value4.append(  sigma_i * (  np.sqrt(  p * np.log(t) + 2 * np.log(1/t**2)   ) + np.sqrt(lbd) * sigma_i ) )

plt.plot( range(9999), value3, label = 'width3')
plt.plot( range(9999), value4, label = 'width4')
plt.legend()

In [ ]:
size = 10
w = np.array([1/size]*size)
context_generator = synthetic_data.LinearContexts( w,'balanced' )

dis = []
for _ in range(1000):
    c,d = context_generator.get_context()
    dis.append(d[0])

plt.hist(dis, bins=30, density=True, alpha=0.7)

In [2]:
import neuralcbpside_v1
import neuralcbpside_v2
import neuralcbpside_v3
import games
import synthetic_data
import numpy as np
import geometry_v3
import cbpside

horizon = 1000
lbd = 0.05
game = games.apple_tasting( )

# size = 5
# w = np.array([1/size]*size)
# context_generator = synthetic_data.LinearContexts( w,'imbalanced' )

w = np.array([0,1])
context_generator = ToyContexts()

# alg = cbpside.CBPside( game, len(w), 1.01, lbd )
# eval1 = Evaluation(None, None, None, horizon, game, None, None)
# job1 = context_generator, 0
# cumRegret1 = eval1.eval_policy_once(alg, game, job1)

alg = neuralcbpside_v3.NeuralCBPside(game, len(w), 1.01, lbd, 10 )
eval2 = Evaluation(None, None, None, horizon, game, None, None)
job2 = context_generator, 0
cumRegret3 = eval2.eval_policy_once(alg, game, job2)

Restricted license - for non-production use only - expires 2024-10-28
0
t 0 action 1 outcome 0 gaps 1

t 1 action 0 outcome 0 gaps 1

factor 7.529243861217872 width [[0.02646047]]
factor 15.505701317935703 width [[0.0187721]]
confidence [array([[0.]]), array([[0.]])]
estimates [array([[0.39148384]], dtype=float32), array([[0.5428325 ],
       [0.03223296]], dtype=float32)]
pair [0, 1] tdelta -0.5105995312333107 confidence 0.0
union1 [0]
t 2 action 0 outcome 0 gaps 1

factor 9.421060044570233 width [[0.04145112]]
factor 19.289333684640425 width [[0.02356977]]
confidence [array([[0.]]), array([[0.]])]
estimates [array([[0.3278068]], dtype=float32), array([[0.33756807],
       [0.05351092]], dtype=float32)]
pair [0, 1] tdelta -0.2840571515262127 confidence 0.0
union1 [0]
t 3 action 1 outcome 1 gaps -1

factor 10.555337814681824 width [[0.04862214]]
factor 21.557889224863608 width [[0.02145883]]
confidence [array([[0.]]), array([[0.]])]
estimates [array([[0.6355209]], dtype=float32), array

KeyboardInterrupt: 

In [15]:

import torch
import torch.nn as nn
import torch.optim as optim

# class Network(nn.Module):
#     def __init__(self, output_dim, dim, hidden_size=10):
#         super(Network, self).__init__()
#         self.fc1 = nn.Linear(dim, hidden_size)
#         self.activate = nn.ReLU()
#         self.fc2 = nn.Linear(hidden_size, output_dim)
#     def forward(self, x):
#         return self.fc2(self.activate(self.fc1(x)))

class Network(nn.Module):
    def __init__(self, output_dim, dim, hidden_size=10):
        super(Network, self).__init__()
        self.fc1 = nn.Linear(dim, hidden_size)
        self.activate = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_dim)
        self.sigmoid = nn.Sigmoid( )
    def forward(self, x):
        x = self.fc2(self.activate(self.fc1(x)))
        x = self.sigmoid(x)
        return x

# Create an instance of your model
model = Network(1, 15).cuda()

# Define your loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01) #, weight_decay=1

length = labels.shape[0]
index = np.arange(length)
np.random.shuffle(index)

epochs = 1000
for epoch in range(epochs):
    running_loss = 0.0
    for i in index:
        input, label = torch.tensor(features[i], dtype=torch.float).cuda() ,  torch.tensor(labels[i], dtype=torch.float).cuda() 
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        output = model(input)
        print(output)
        # loss = nn.MSELoss()(output, label)
        loss = nn.BCELoss()(output, label)
        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Track the loss
        running_loss += loss.item()

    # Print the average loss for the epoch
    print(f"Epoch {epoch + 1} Loss: {running_loss }")

print("Training finished.")

tensor([0.5344], device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([0.5329], device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([0.5376], device='cuda:0', grad_fn=<SigmoidBackward0>)
Epoch 1 Loss: 2.1651379466056824
tensor([0.5327], device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([0.5312], device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([0.5357], device='cuda:0', grad_fn=<SigmoidBackward0>)
Epoch 2 Loss: 2.160540997982025
tensor([0.5309], device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([0.5296], device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([0.5339], device='cuda:0', grad_fn=<SigmoidBackward0>)
Epoch 3 Loss: 2.156024694442749
tensor([0.5292], device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([0.5279], device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([0.5321], device='cuda:0', grad_fn=<SigmoidBackward0>)
Epoch 4 Loss: 2.1515873670578003
tensor([0.5275], device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([0.5262], device='cuda:0', grad_fn=<SigmoidBackward0>)
te

In [ ]:
# plt.plot(range(horizon), np.cumsum(cumRegret1) )
# plt.plot(range(horizon), np.cumsum(cumRegret2) )
plt.plot(range(horizon), np.cumsum(cumRegret3) )

In [12]:
features = None
labels = None

In [13]:
import numpy as np

class LinearContexts:
    def __init__(self,  w, task):
        self.d = len(w) # number of features
        self.w = w
        self.type = 'linear'
        self.task = task
    
    def get_context(self, ):

        if self.task == 'imbalanced':
            context = truncated_gaussian(0, 0.1, 0, 1, self.d) if np.random.uniform(0,1)<0.5 else truncated_gaussian(1, 0.1, 0, 1, self.d)
        else:
            context = truncated_gaussian(0.5, 1, 0, 1, self.d)
        # cont = context.reshape(self.d,1)
        p = self.w @ context
        val = [ p, 1-p ]
        context = context - np.ones(5) * 0.5
        context = context.astype(np.float64)
        return context , val 

A = 3
size = 5
w = np.array([1/size]*size)
context_generator = LinearContexts(w, 'imbalanced')

for _ in range(1):
    c,d = context_generator.get_context()
    c = reshape_context(c, A)

    e_y = np.zeros( A )
    outcome = np.random.choice( 2 , p = d )
    e_y[outcome] = 1
    Y_t = e_y.reshape(A,1)
    features = c if features is None else np.concatenate((features, c), axis=0)
    labels = Y_t if labels is None else np.concatenate((labels, Y_t), axis=0)


In [33]:
c = np.empty((0,5*3,1))
c1,d1 = context_generator.get_context()
c2,d2 = context_generator.get_context()
a1 = reshape_context(c1, 3)
a2 = reshape_context(c2, 3)

history = None


w = np.array([1/size]*size)